In [45]:
import numpy as np
import pandas as pd
import pickle
import os
import sqlite3
from pandas.io import sql

In [3]:
! ls "../../../datasets/lending-club/"

LCDataDictionary.xlsx loans                 rejects


In [8]:
path = "../../../datasets/lending-club/"

In [9]:
loan_files = os.listdir(path+"loans/")
loan_files

['LoanStats3a.csv',
 'LoanStats3b.csv',
 'LoanStats3c.csv',
 'LoanStats3d.csv',
 'LoanStats_2016Q1.csv',
 'LoanStats_2016Q2.csv',
 'LoanStats_2016Q3.csv',
 'LoanStats_2016Q4.csv']

In [10]:
reject_files = os.listdir(path+"rejects/")
reject_files

['RejectStats_2016Q1.csv',
 'RejectStats_2016Q2.csv',
 'RejectStats_2016Q3.csv',
 'RejectStats_2016Q4.csv',
 'RejectStatsA.csv',
 'RejectStatsB.csv',
 'RejectStatsD.csv']

In [18]:
def read_file(name, lr, item):
    return pd.read_csv(path+name+lr[0], low_memory=False, skiprows=1)

# Loans

In [25]:
len(loan_files)

8

In [26]:
df1 = read_file("loans/", loan_files, 0)
df2 = read_file("loans/", loan_files, 1)
df3 = read_file("loans/", loan_files, 2)
df4 = read_file("loans/", loan_files, 3)
df5 = read_file("loans/", loan_files, 4)
df6 = read_file("loans/", loan_files, 5)
df7 = read_file("loans/", loan_files, 6)
df8 = read_file("loans/", loan_files, 7)

In [38]:
# CHECK THAT OUR COLUMNS ALL HAVE THE SAME NAME AND ARE IN THE SAME ORDER

# SET REMOVES DUPLICATES
cols = [list(set(i)) for i in zip(df1.columns, df2.columns, df3.columns, df4.columns, 
           df5.columns, df6.columns, df7.columns, df8.columns)]

# CHECK THAT EACH ELEMENT IN COLS ONLY HAS ONE ELEMENT IN IT
all([len(i) == 1 for i in cols])

True

In [43]:
df1.shape[0]+df2.shape[0]+df3.shape[0]+df4.shape[0]+df5.shape[0]+df6.shape[0]+df7.shape[0]+df8.shape[0]

340304

In [39]:
loan_data = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])

In [41]:
loan_data.shape

(340304, 111)

In [50]:
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340304 entries, 0 to 42537
Columns: 111 entries, id to total_il_high_credit_limit
dtypes: float64(87), object(24)
memory usage: 290.8+ MB


In [44]:
with open("loan_data.pkl", "wb") as f:
    pickle.dump(loan_data, f)

In [46]:
sqlite_db = 'loan_db.sqlite'
conn = sqlite3.connect(sqlite_db)

In [48]:
loan_data.to_sql("loan_data", 
                 con=conn, 
                 if_exists="replace", 
                 index=False)

In [49]:
sql.read_sql("select * from loan_data limit 5", con=conn)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,None,None,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,None,None,None,None,0.0,0.0,None,None,None,None
1,None,None,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,None,None,None,None,0.0,0.0,None,None,None,None
2,None,None,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,None,None,None,None,0.0,0.0,None,None,None,None
3,None,None,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,None,None,None,None,0.0,0.0,None,None,None,None
4,None,None,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,None,None,None,None,0.0,0.0,None,None,None,None


## Rejections

In [57]:
len(reject_files)

7

In [60]:
r1 = read_file("rejects/", reject_files, 0)
r2 = read_file("rejects/", reject_files, 1)
r3 = read_file("rejects/", reject_files, 2)
r4 = read_file("rejects/", reject_files, 3)
r5 = read_file("rejects/", reject_files, 4)
r6 = read_file("rejects/", reject_files, 5)
r7 = read_file("rejects/", reject_files, 6)

In [62]:
# CHECK THAT OUR COLUMNS ALL HAVE THE SAME NAME AND ARE IN THE SAME ORDER

# SET REMOVES DUPLICATES
rcols = [list(set(i)) for i in zip(r1.columns, r2.columns, r3.columns, 
                                   r4.columns, r5.columns, r6.columns, r7.columns)]

# CHECK THAT EACH ELEMENT IN COLS ONLY HAS ONE ELEMENT IN IT
all([len(i) == 1 for i in rcols])

True

In [61]:
r1.shape[0]+r2.shape[0]+r3.shape[0]+r4.shape[0]+r5.shape[0]+r6.shape[0]+r7.shape[0]

7673428

In [63]:
reject_data = pd.concat([r1,r2,r3,r4,r5,r6,r7])

In [68]:
reject_data.columns = [i.lower().replace(" ", "_").replace("-", "_") for i in reject_data.columns]

In [69]:
reject_data.head()

,amount_requested,application_date,loan_title,risk_score,debt_to_income_ratio,zip_code,state,employment_length,policy_code
0,35000.0,2016-01-01,credit_card,NaN,100%,708xx,LA,< 1 year,0
1,7500.0,2016-01-01,debt_consolidation,NaN,1.52%,423xx,KY,< 1 year,0
2,25000.0,2016-01-01,debt_consolidation,NaN,95.85%,731xx,OK,< 1 year,0
3,3500.0,2016-01-01,moving,NaN,10.12%,068xx,CT,< 1 year,0
4,35000.0,2016-01-01,credit_card,621.0,1867.45%,770xx,TX,< 1 year,0


In [70]:
with open("reject_data.pkl", "wb") as f:
    pickle.dump(reject_data, f)

In [71]:
reject_data.to_sql("reject_data", 
                 con=conn, 
                 if_exists="replace", 
                 index=False)

In [72]:
sql.read_sql("select * from reject_data limit 5", con=conn)

,amount_requested,application_date,loan_title,risk_score,debt_to_income_ratio,zip_code,state,employment_length,policy_code
0,35000.0,2016-01-01,credit_card,NaN,100%,708xx,LA,< 1 year,0
1,7500.0,2016-01-01,debt_consolidation,NaN,1.52%,423xx,KY,< 1 year,0
2,25000.0,2016-01-01,debt_consolidation,NaN,95.85%,731xx,OK,< 1 year,0
3,3500.0,2016-01-01,moving,NaN,10.12%,068xx,CT,< 1 year,0
4,35000.0,2016-01-01,credit_card,621.0,1867.45%,770xx,TX,< 1 year,0
